In [ ]:
# [준비 단계]
# 의료 데이터셋을 활용하여, DenseNet 모델에 대한 성능을 확인합니다.

# >> 비교 모델명        | 역할              | 비교 관점
# ResNet-50 (Baseline) | 표준 기준         | 산업계 표준 모델과의 성능 대비 효율성 확인
# DenseNet-121 (k=32)  | 효율성 대표	    | 가장 적은 자원으로 ResNet-50 수준의 성능 달성 여부
# DenseNet-169 (k=32)  | 깊이 중심 밀집 연결 | 층을 깊게 쌓았을 때(k=32)의 특징 추출 능력
# DenseNet-161 (k=48)  | 너비 중심 밀집 연결 | 성장률(k=48)을 키웠을 때의 성능 극대화 확인

# 데이터셋 : The IQ-OTH_NCCD lung cancer dataset (폐암)
# https://www.kaggle.com/datasets/hamdallak/the-iqothnccd-lung-cancer-dataset/code

# [중요]
# 다운 받은 데이터셋을 root 폴더 내 'Data' 폴더에 압축 해제하여 3가지 폴더에 각각 이미지들이 있는 것을 확인할 것.
# Data/Benign cases     ; 여기서 직접 폴더명의 오타를 수정한다. Benign -> Benign
# Data/Malignant cases
# Data/Normal cases

# 3가지 클래스 정보
# Normal(정상)
# Malignant (악성) : 악성 종양, 즉 암. 
# Benign (양성) : 양성 종양, 즉 암이 아닌 종양.

In [ ]:
# [Cell 1] 설정 : 시작 import 라이브러리, 환경, 전역 변수
#  Epoch, 데이터 분할 비율 등 핵심 설정을 여기서 한 번에 관리합니다.

import os
import cv2
import time
import random
import shutil
import platform
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score, f1_score
from tqdm.auto import tqdm
from PIL import Image

import gc # [수정] 가비지 컬렉터 임포트 추가

# ----------------------------------------
# [Global Configurations] 핵심 설정
# ----------------------------------------
# 1. 학습 관련 변수 (여기서 수정하면 전체 반영)
EPOCHS = 15 # min 5 for checking, best 15, max 30

# 224x224 인 경우, Batch 32~16 로
# 448x448 인 경우, Batch 16~8 로
size = 224 
BATCH_SIZE = 16 

# 2. 데이터 분할 비율 (Train : Valid : Test)
# 합이 1.0이 되도록 설정!! train 70% 상황에서 최대한 valid와 test의 균형을 맞춘다.
SPLIT_RATIO = (0.7, 0.15, 0.15) 

# 3. 시드 고정 -> 재현성 확보
SEED = 2026

# 결과 저장 폴더 정의
FOLDERS = {
    'visuals': './Result_Visuals_Tight',    # 마스킹 전처리 검증용
    'results': './Result_Tight',            # 학습 결과(CSV, Model)
    'gradcam': './Result_GradCAM_Tight'     # Grad-CAM 결과 -> 모델의 가중치 영역 시각화 (*.pth 관련)
}

# 폴더 자동 생성
for k, v in FOLDERS.items():
    os.makedirs(v, exist_ok=True)

# 시드 고정 및 장치 설정
def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# 로컬 PC의 GPU 사용 설정
set_seed()
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f">> 환경 설정 완료.")
print(f">> Device: {DEVICE}")
print(f">> Epochs: {EPOCHS}")
print(f">> Split Ratio: {SPLIT_RATIO}")

In [ ]:
# [Cell 2] 데이터셋 분할 (Data Split)
# 설명: 원본 Data 폴더에서 SPLIT_RATIO에 맞춰 Data_split 폴더로 데이터를 무작위 분할합니다.

# 원본 데이터 경로 (VSCode 작업 폴더 기준)
DATA_DIR = './Data'

# 분할된 데이터 저장 경로
SPLIT_DIR = './Data_split'

def split_dataset():
    # 이미 분할된 데이터가 있는지 확인
    if os.path.exists(SPLIT_DIR) and len(glob.glob(os.path.join(SPLIT_DIR, 'train', '*', '*'))) > 0:
        print(">> 데이터셋이 이미 분할되어 있습니다. 기존 데이터를 사용합니다.")
        return

    print(f">> 데이터 분할 시작 (비율 {SPLIT_RATIO})...")
    
    # 기존 분할 폴더가 있다면 삭제 후 재생성
    if os.path.exists(SPLIT_DIR): shutil.rmtree(SPLIT_DIR)
    
    # 클래스 폴더 읽기
    classes = [d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))]
    
    for cls in classes:
        # Train/Valid/Test 폴더 생성
        for split in ['train', 'valid', 'test']:
            os.makedirs(os.path.join(SPLIT_DIR, split, cls), exist_ok=True)
        
        # 이미지 파일 리스트 확보
        src_path = os.path.join(DATA_DIR, cls)
        files = []
        for ext in ['*.jpg', '*.png', '*.jpeg', '*.JPG', '*.PNG']:
            files.extend(glob.glob(os.path.join(src_path, ext)))
            
        # 셔플 및 인덱스 계산
        random.shuffle(files)
        total = len(files)
        train_end = int(total * SPLIT_RATIO[0])
        valid_end = int(total * (SPLIT_RATIO[0] + SPLIT_RATIO[1]))
        
        # 파일 복사
        for f in files[:train_end]: 
            shutil.copy(f, os.path.join(SPLIT_DIR, 'train', cls, os.path.basename(f)))
        for f in files[train_end:valid_end]: 
            shutil.copy(f, os.path.join(SPLIT_DIR, 'valid', cls, os.path.basename(f)))
        for f in files[valid_end:]: 
            shutil.copy(f, os.path.join(SPLIT_DIR, 'test', cls, os.path.basename(f)))
    
    print(">> 데이터 분할 및 복사 완료.")

split_dataset()

In [ ]:
# [Cell 3] 정밀 마스킹(몸통 안쪽만) 및 데이터 증강 (반전/회전)
# - 각 이미지에 대해서 몸통 바깥쪽의 정보를 배제하고 몸통 안쪽을 학습하기 위해서.
# - 경계 안쪽만 남기는 정밀 마스킹을 수행한다.
# - 대체로 이미지의 가운데에 위치한 폐 영역에 적용할 적절한 증강 방법으로 반전 및 회전을 적용합니다.

import cv2
import numpy as np
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import glob
import matplotlib.pyplot as plt
import random

# --- 1. 정밀 마스킹 함수 (몸통 경계 안쪽만 남기기) ---
def preprocess_tight_body(image_path, visualize=False):
    # 이미지 로드
    stream = open(image_path, "rb")
    bytes = bytearray(stream.read())
    numpy_array = np.asarray(bytes, dtype=np.uint8)
    img_array = cv2.imdecode(numpy_array, cv2.IMREAD_UNCHANGED)
    stream.close()
    
    if img_array is None: return None
    
    # 3채널 보장
    if len(img_array.shape) == 2: 
        img_array = cv2.cvtColor(img_array, cv2.COLOR_GRAY2BGR)
    elif img_array.shape[2] == 4: 
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGRA2BGR)
    
    # BGR -> Gray 변환
    gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
    
    # -------------------------------------------------------------------------
    # [핵심 로직] 몸통 경계 찾기 및 배경 제거
    # -------------------------------------------------------------------------
    # 노이즈 제거 및 이진화 (Otsu Algorithm)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # 구멍 메우기 (폐 내부의 공기층도 몸통으로 인식되게 함)
    kernel = np.ones((5,5), np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
    
    # 윤곽선(Contour) 찾기
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 컨투어를 못 찾으면 원본 반환 (안전장치)
    if not contours:
        return Image.fromarray(cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB))
    
    # 가장 큰 영역 = 몸통
    max_contour = max(contours, key=cv2.contourArea)
    
    # 마스크 생성 (몸통 안쪽은 255, 바깥쪽은 0)
    mask = np.zeros_like(gray)
    cv2.drawContours(mask, [max_contour], -1, 255, -1)
    
    # 마스킹 적용 (배경을 검은색으로 지움)
    masked_img = cv2.bitwise_and(img_array, img_array, mask=mask)
    
    # 몸통 경계선 시각화용: 노란색 라인 그리기
    if visualize:
        vis_img = masked_img.copy()
        # 노란색 (BGR: 0, 255, 255)
        cv2.drawContours(vis_img, [max_contour], -1, (0, 255, 255), 3)
        return cv2.cvtColor(vis_img, cv2.COLOR_BGR2RGB)
    
    # Crop (몸통 영역만 최대한 타이트하게 잘라냄)
    x, y, w, h = cv2.boundingRect(max_contour)

    # 약간의 여백(Padding) 추가하여 경계선 정보 보존
    pad = 10
    x = max(0, x-pad); y = max(0, y-pad)
    w = min(img_array.shape[1]-x, w+2*pad)
    h = min(img_array.shape[0]-y, h+2*pad)
    
    final_img = masked_img[y:y+h, x:x+w]
    
    # BGR -> RGB 변환 후 PIL Image로 반환
    return Image.fromarray(cv2.cvtColor(final_img, cv2.COLOR_BGR2RGB))


# --- 2. Dataset 정의 ---
class TightLungDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []
        if os.path.exists(root_dir):
            self.class_names = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
            self.class_to_idx = {cls: i for i, cls in enumerate(self.class_names)}
            
            for cls in self.class_names:
                cls_folder = os.path.join(root_dir, cls)
                for f in os.listdir(cls_folder):
                    if f.lower().endswith(('.png', '.jpg', '.jpeg')):
                        self.samples.append((os.path.join(cls_folder, f), self.class_to_idx[cls]))
        else:
            self.class_names = []

    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        path, label = self.samples[idx]
        # 배경 제거된 몸통 이미지 가져오기
        image = preprocess_tight_body(path, visualize=False)
        if self.transform: image = self.transform(image)
        return image, label


# --- 3. 데이터 증강 및 로더 설정 ---
# 
# 첫 번째 셀 상단에서 조정.
# size = 224 # 기본값
# BATCH_SIZE = 16 # 기본값

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((size, size)),
        
        # 폐 영역의 반전과 회전만 적용 (이동 제외)
        transforms.RandomHorizontalFlip(p=0.5),      # 좌우 반전
        transforms.RandomRotation(degrees=15),       # 회전 (+-15도)
        
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    
    'valid': transforms.Compose([
        transforms.Resize((size, size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    
    'test': transforms.Compose([
        transforms.Resize((size, size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

dataloaders = {
    'train': DataLoader(TightLungDataset(os.path.join(SPLIT_DIR, 'train'), transform=data_transforms['train']),
                        batch_size=BATCH_SIZE, shuffle=True),
    'valid': DataLoader(TightLungDataset(os.path.join(SPLIT_DIR, 'valid'), transform=data_transforms['valid']),
                        batch_size=BATCH_SIZE, shuffle=False),
    'test':  DataLoader(TightLungDataset(os.path.join(SPLIT_DIR, 'test'), transform=data_transforms['test']),
                        batch_size=BATCH_SIZE, shuffle=False)
}

class_names = dataloaders['train'].dataset.class_names
print(f">> Dataset 준비 완료.")
print(f">> Preprocessing: Tight Body Masking (Background Removed)")
print(f">> Augmentation: Flip, Rotate(15) ONLY")
print(f">> Resolution: {size}x{size} | Batch: {BATCH_SIZE}")


# --- [검증] 클래스별 전처리 및 증강 통합 확인 ---
def verify_preprocessing_with_aug():
    print(">> [전처리/증강 검증] 클래스별 샘플 확인 (상단: 마스킹, 하단: 증강 적용)...")
    
    target_classes = class_names  # ['Normal', 'Malignant', 'Benign']
    num_classes = len(target_classes)
    
    # 증강 시뮬레이션용 transform (ToTensor 제외하고 PIL 상태로 확인)
    aug_sim = transforms.Compose([
        transforms.Resize((size, size)),
        transforms.RandomHorizontalFlip(p=1.0),  # 확인을 위해 강제 반전
        transforms.RandomRotation(degrees=15)    # 15도 회전
    ])
    
    fig, axes = plt.subplots(2, num_classes, figsize=(6 * num_classes, 10))
    
    for i, cls in enumerate(target_classes):
        cls_path = os.path.join(SPLIT_DIR, 'test', cls)
        if not os.path.exists(cls_path):
            cls_path = os.path.join(DATA_DIR, cls)
            
        img_list = glob.glob(os.path.join(cls_path, '*'))
        if not img_list: continue
            
        sample_path = random.choice(img_list)
        
        # 1. 배경 제거 및 경계선 시각화 (원본 마스킹 확인)
        img_vis = preprocess_tight_body(sample_path, visualize=True)
        
        # 2. 깨끗한 마스킹 이미지에 증강 적용
        img_clean = preprocess_tight_body(sample_path, visualize=False)
        img_aug = aug_sim(img_clean)
        
        fname = os.path.basename(sample_path)
        
        # 상단: 마스킹 결과 (노란선)
        axes[0, i].imshow(img_vis)
        axes[0, i].set_title(f"Class: {cls}\n[Masking & Crop]", fontsize=12, fontweight='bold')
        axes[0, i].axis('off')
        
        # 하단: 증강 결과 (반전/회전 적용)
        axes[1, i].imshow(img_aug)
        axes[1, i].set_title(f"Augmented (Flip+Rotate)\n{size}x{size}", fontsize=12)
        axes[1, i].axis('off')
        
        # 개별 저장
        plt.imsave(os.path.join(FOLDERS['visuals'], f'verify_{cls}_masked.png'), img_vis)
        plt.imsave(os.path.join(FOLDERS['visuals'], f'verify_{cls}_aug.png'), np.array(img_aug))

    plt.tight_layout()
    plt.show()

# 실행
verify_preprocessing_with_aug()

In [ ]:
# [Cell 4] 모델 생성 팩토리 클래스 정의

class ModelFactory:
    @staticmethod
    def get_model(model_name, num_classes):
        
        if model_name == 'resnet50':
            model = models.resnet50(weights='DEFAULT')
            model.fc = nn.Linear(model.fc.in_features, num_classes)
        elif model_name == 'densenet121':
            model = models.densenet121(weights='DEFAULT')
            model.classifier = nn.Linear(model.classifier.in_features, num_classes)
        elif model_name == 'densenet169':
            model = models.densenet169(weights='DEFAULT')
            model.classifier = nn.Linear(model.classifier.in_features, num_classes)
        elif model_name == 'densenet161':
            model = models.densenet161(weights='DEFAULT')
            model.classifier = nn.Linear(model.classifier.in_features, num_classes)
            
        else:
            raise ValueError(f"Unknown model: {model_name}")
        
        return model.to(DEVICE)

In [ ]:
# [Cell 5] 모델 학습 및 상세 지표 CSV 저장 (안전한 중간 저장 기능 추가)
# 설명: 모델별로 학습이 끝날 때마다 개별 CSV를 저장하여, 중간에 끊겨도 기록을 보존합니다.

def train_and_log_models(epochs):
    print(f"\n=== 정밀 마스킹 모델 학습 시작 (Epochs: {epochs}) ===")
    
    models_list = ['resnet50', 'densenet121', 'densenet169', 'densenet161']
    
    # 전체 기록을 담을 리스트 (마지막 통합 저장용)
    all_history = []
    
    overall_pbar = tqdm(models_list, desc="Total Progress", position=0)
    
    for name in overall_pbar:
        print(f"\n>> Training Model: {name}")
        
        # [NEW] 현재 모델의 기록만 담을 리스트 (개별 저장용)
        model_history = []
        # ModelFactory, class_names, DEVICE 등은 이전 셀에서 정의된 전역 변수 사용
        model = ModelFactory.get_model(name, len(class_names))
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=1e-4)
        
        # 스케줄러 정의
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-6)
        
        best_f1 = 0.0
        
        for ep in range(epochs):
            start_time = time.time()
            
            # --- Train ---
            model.train()
            running_train_loss = 0.0

            # dataloaders는 이전 셀에서 정의된 전역 변수 사용
            for inputs, labels in tqdm(dataloaders['train'], desc=f"Ep {ep+1}", leave=False):
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_train_loss += loss.item() * inputs.size(0)
            
            # 스케줄러 업데이트 및 LR 기록
            scheduler.step()
            current_lr = optimizer.param_groups[0]['lr']
            
            epoch_train_loss = running_train_loss / len(dataloaders['train'].dataset)
            
            # --- Valid ---
            model.eval()
            running_valid_loss = 0.0
            all_preds, all_labels = [], []
            with torch.no_grad():
                for inputs, labels in dataloaders['valid']:
                    inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    running_valid_loss += loss.item() * inputs.size(0)
                    _, preds = torch.max(outputs, 1)
                    all_preds.extend(preds.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())
            
            epoch_valid_loss = running_valid_loss / len(dataloaders['valid'].dataset)
            
            # Metrics
            acc = accuracy_score(all_labels, all_preds)
            precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted', zero_division=0)
            
            cm = confusion_matrix(all_labels, all_preds)
            TP = np.trace(cm); FP = cm.sum(axis=0) - np.diag(cm)
            FN = cm.sum(axis=1) - np.diag(cm); TN = cm.sum() - (FP + FN + TP)
            TP, FP, FN, TN = int(TP), int(FP.sum()), int(FN.sum()), int(TN.sum())

            duration = time.time() - start_time
            print(f"Ep {ep+1}/{epochs} | LR: {current_lr:.6f} | T-Loss: {epoch_train_loss:.4f} | V-Loss: {epoch_valid_loss:.4f} | Acc: {acc:.4f} | F1: {f1:.4f} | Time: {duration:.1f}s")
            
            # 기록 데이터 생성 -> 별도의 셀에서 csv 파일의 결과값들을 사용하여 그래프로 표현.
            record = {
                'model': name, 'epoch': ep + 1, 'learning_rate': current_lr,
                'train_loss': epoch_train_loss, 'valid_loss': epoch_valid_loss,
                'TP': TP, 'TN': TN, 'FP': FP, 'FN': FN,
                'Accuracy': acc, 'Precision': precision, 'Recall': recall, 'F1_score': f1,
                'Time_sec': duration
            }
            
            model_history.append(record) # 개별 리스트에 추가
            all_history.append(record)   # 전체 리스트에 추가
            
            if f1 > best_f1:
                best_f1 = f1
                torch.save(model.state_dict(), os.path.join(FOLDERS['results'], f'best_{name}.pth'))
        
        # [NEW] 모델 하나가 끝날 때마다 개별 CSV 저장
        df_model = pd.DataFrame(model_history)
        model_csv_path = os.path.join(FOLDERS['results'], f'metrics_{name}.csv')
        df_model.to_csv(model_csv_path, index=False)
        print(f">> [저장 완료] {name} 학습 기록 저장됨: {model_csv_path}")

        # --- [VRAM 초기화 강화 섹션] ---
        # 1. 모델과 관련된 모든 객체 참조 끊기
        try:
            del model
            del optimizer
            del criterion
            del scheduler
            if 'inputs' in locals(): del inputs
            if 'labels' in locals(): del labels
            if 'outputs' in locals(): del outputs
        except NameError:
            pass

        # 2. Python 가비지 컬렉션 강제 실행
        gc.collect() 

        # 3. PyTorch의 CUDA 캐시 비우기 (GPU 메모리 해제)
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize() # 모든 스트림의 작업이 끝날 때까지 대기하여 확실히 비움
            
        print(f">> {name} Finished. Best F1: {best_f1:.4f} (VRAM Cleared)")
        print("-" * 80)

    # 마지막에 전체 통합 파일도 저장 (분석용)
    df_all = pd.DataFrame(all_history)
    all_csv_path = os.path.join(FOLDERS['results'], 'metrics_all_details.csv')
    df_all.to_csv(all_csv_path, index=False)
    print(f"\n>> [최종 저장] 모든 모델 통합 기록 저장됨: {all_csv_path}")

# 실행
train_and_log_models(epochs=EPOCHS)

In [ ]:
# [Cell 6] 학습 결과의 그래프 시각화 (스타일 적용 & 안전성 강화)
# 각 모델별 Train/Valid Loss, F1-score, 누적 시간 비교

import matplotlib.ticker as ticker
import pandas as pd
import os
import matplotlib.pyplot as plt

def plot_metrics_comparison_2x2_styled():
    # 비교할 모델 리스트
    models = ['resnet50', 'densenet121', 'densenet169', 'densenet161']
    
    # 개별 CSV 파일들을 읽어서 하나로 합치기
    df_list = []
    print(">> 데이터 로드 중...")
    
    for name in models:
        csv_path = os.path.join(FOLDERS['results'], f'metrics_{name}.csv')
        if os.path.exists(csv_path):
            df = pd.read_csv(csv_path)
            df_list.append(df)
            print(f"   - {name}: 로드 완료 ({len(df)} Epochs)")
        else:
            print(f"   - {name}: 파일 없음 (Skipping)")
            
    if not df_list:
        print("!! 로드할 데이터가 없습니다. 학습을 먼저 실행하세요.")
        return

    # 데이터프레임 통합
    df_all = pd.concat(df_list, ignore_index=True)
    
    # 누적 시간(Cumulative Time) 계산
    # Time_sec가 에포크별 시간이므로, 누적합(cumsum)을 구해야 총 학습 시간이 됨
    df_all['Cumulative_Time'] = df_all.groupby('model')['Time_sec'].cumsum()
    
    # [수정] 데이터 기반 최대 에포크 확인 (EPOCHS 변수 의존성 제거)
    max_epoch = df_all['epoch'].max()
    
    # 2행 2열 그래프 생성
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 그래프 배치 설정 (행, 열, CSV컬럼명, 제목, Y축라벨)
    plots_config = [
        (0, 0, 'train_loss',      'Train Loss',           'Loss'),
        (0, 1, 'valid_loss',      'Valid Loss',           'Loss'),
        (1, 0, 'F1_score',        'Validation F1-score',  'F1-score'),     
        (1, 1, 'Cumulative_Time', 'Cumulative Training Time', 'Time (sec)') 
    ]
    
    # 라벨창 스타일 지정 (가시성 강화)
    styles = {
        'resnet50':    {'marker': 's', 'linestyle': '--', 'color': 'navy',   'label': 'ResNet50'},
        'densenet121': {'marker': '^', 'linestyle': '-',  'color': 'orange', 'label': 'DenseNet121'},
        'densenet169': {'marker': 'D', 'linestyle': '-',  'color': 'red',    'label': 'DenseNet169'},
        'densenet161': {'marker': 'o', 'linestyle': '-',  'color': 'brown',  'label': 'DenseNet161'}
    }

    print(">> 그래프 생성 시작...")

    # 데이터 플로팅
    for name in models:
        df = df_all[df_all['model'] == name]
        if df.empty: continue

        st = styles.get(name, {'marker': 'o', 'linestyle': '-', 'color': 'black'})
        
        for r, c, col_name, title, ylabel in plots_config:
            ax = axes[r, c]
            ax.plot(df['epoch'], df[col_name], 
                    marker=st['marker'], 
                    linestyle=st['linestyle'],
                    color=st['color'],
                    label=st.get('label', name), 
                    linewidth=2,
                    alpha=0.8)

    # 라벨창 스타일 그리기 (가시성 강화)
    for r, c, col_name, title, ylabel in plots_config:
        ax = axes[r, c]
        
        # 타이틀 및 라벨
        ax.set_title(title, fontsize=22, fontweight='bold', pad=20)
        ax.set_xlabel('Epochs', fontsize=20, fontweight='bold', labelpad=10)
        ax.set_ylabel(ylabel, fontsize=20, fontweight='bold', labelpad=10)

        # Y축 범위 설정 추가 구간]
        if 'Loss' in ylabel:            # Loss 그래프 공통 (Train & Valid)
            ax.set_ylim(-0.02, 0.62)
        elif 'F1-score' in ylabel:      # F1-score 그래프
            ax.set_ylim(0.79, 1.01)

        #  눈금 설정
        ax.tick_params(axis='both', which='major', labelsize=14, pad=10, width=2)
        for label in ax.get_xticklabels() + ax.get_yticklabels():
            label.set_fontweight('bold')
        
        # 그리드 및 테두리
        ax.grid(True, alpha=0.3, linestyle='--', linewidth=1.5)
        for spine in ax.spines.values():
            spine.set_linewidth(2)

        # 범례 위치 로직
        if 'Loss' in title:   # 1행 (Loss) -> 우측 상단
            legend_loc = 'upper right'
        elif 'F1' in title:   # 2행 1열 (F1) -> 우측 하단
            legend_loc = 'lower right'
        else:                 # 2행 2열 (Time) -> 좌측 상단
            legend_loc = 'upper left'

        ax.legend(fontsize=14, loc=legend_loc, prop={'weight':'bold', 'size':14})

        # x축 간격 설정 (5단위)
        ax.xaxis.set_major_locator(ticker.MultipleLocator(5))
        
        # [수정] x축 범위 (데이터 기반)
        ax.set_xlim(0, max_epoch + 1) 

        # Y축 범위 설정 (Time은 0부터)
        if 'Time' in title:
            ax.set_ylim(bottom=0) 

    plt.tight_layout()
    plt.subplots_adjust(hspace=0.4, wspace=0.3) # 간격 조정
    
    # 저장 및 출력
    save_path = os.path.join(FOLDERS['results'], 'metrics_comparison_2x2_styled.png')
    plt.savefig(save_path, dpi=300)
    plt.show()
    print(f">> 그래프 저장 완료: {save_path}")

# 실행
plot_metrics_comparison_2x2_styled()

In [ ]:
# [Cell 7] Confusion Matrix 시각화 (2x2 Grid)
# Actual vs Predicted
# 설명: 저장된 Best 모델(pth)을 불러와 Test 셋에 대해 추론하고, Confusion Matrix를 2행 2열로 그려서 비교합니다.

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch
import os
import platform # [추가] 시스템 확인용
from matplotlib import rc
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score # [추가] f1_score

# --- 한글 폰트 설정 함수 ---
def set_korean_font():
    system_name = platform.system()
    try:
        if system_name == 'Windows':
            rc('font', family='Malgun Gothic')
        elif system_name == 'Darwin': 
            rc('font', family='AppleGothic')
        else: 
            # Colab 등 리눅스 환경 고려
            rc('font', family='NanumBarunGothic') 
    except:
        print("Warning: 폰트 설정에 실패했습니다. 기본 폰트를 사용합니다.")
    
    plt.rcParams['axes.unicode_minus'] = False

set_korean_font()

def plot_confusion_matrices_final_v3():
    print(">> Confusion Matrix 시각화 시작...")
    
    # 2x2 배치 설정 (리스트 사이에 쉼표(,) 추가 필수)
    grid_models = [
        ['densenet121', 'densenet161'],
        ['densenet169', 'resnet50']
    ]
    
    # 컬러맵 설정
    cmaps = [
        ['Oranges', 'Oranges'],
        ['Oranges', 'Blues']  
    ]
    
    # 모델명 매핑
    model_pretty_names = {
        'resnet50': 'ResNet50',
        'densenet121': 'DenseNet121',
        'densenet169': 'DenseNet169',
        'densenet161': 'DenseNet161'
    }
    
    # 라벨 매핑 (Normal -> Malignant -> Benign 순서 정렬)
    target_order = ['Normal', 'Malignant', 'Benign']
    display_names = {
        'Normal': 'Normal (정상)',
        'Malignant': 'Malignant (악성)',
        'Benign': 'Benign (양성)'
    }
    
    # 클래스 이름 매칭 로직
    reorder_idx = []
    final_display_labels = []
    
    # class_names가 정의되어 있는지 확인
    current_class_names = class_names if 'class_names' in globals() else ['Benign', 'Malignant', 'Normal']

    for target in target_order:
        found = [name for name in current_class_names if target.lower() in name.lower()]
        if found:
            original_name = found[0]
            reorder_idx.append(current_class_names.index(original_name))
            final_display_labels.append(display_names[target])
    
    # 매칭 실패 시 기본값 사용
    if len(reorder_idx) != 3:
        reorder_idx = range(len(current_class_names))
        final_display_labels = current_class_names

    # 정답 데이터 로드 (Test Set 전체)
    # Test Loader는 shuffle=False여야 순서가 보장됨
    y_true_raw = []
    for _, y in dataloaders['test']:
        y_true_raw.extend(y.numpy())
    y_true_raw = np.array(y_true_raw)
    
    # 그래프 생성
    fig, axes = plt.subplots(2, 2, figsize=(16, 16))
    
    for row in range(2):
        for col in range(2):
            model_name = grid_models[row][col]
            cmap_name = cmaps[row][col]
            ax = axes[row][col]
            
            # 모델의 학습 영역 가중치 관련 *.pth 파일 경로 확인
            pth_path = os.path.join(FOLDERS['results'], f'best_{model_name}.pth')
            
            if not os.path.exists(pth_path):
                ax.text(0.5, 0.5, "Model Not Found", ha='center', va='center', fontsize=15)
                ax.axis('off')
                continue
            
            # 모델 로드 & 예측
            # ModelFactory는 [Cell 3]에서 정의됨
            try:
                model = ModelFactory.get_model(model_name, len(current_class_names))
                model.load_state_dict(torch.load(pth_path, map_location=DEVICE))
                model.eval()
                model.to(DEVICE)
                
                y_pred_raw = []
                with torch.no_grad():
                    for x, _ in dataloaders['test']:
                        x = x.to(DEVICE)
                        out = model(x)
                        y_pred_raw.extend(out.argmax(1).cpu().numpy())
                
                # 혼동 행렬 계산
                cm_raw = confusion_matrix(y_true_raw, y_pred_raw)
                # 순서 재배치 (Numpy indexing)
                cm_ordered = cm_raw[np.ix_(reorder_idx, reorder_idx)]
                row_sums = cm_ordered.sum(axis=1)
                
                # 히트맵 그리기
                sns.heatmap(cm_ordered, annot=False, fmt='', cmap=cmap_name, ax=ax,
                            xticklabels=final_display_labels, 
                            yticklabels=final_display_labels, 
                            cbar=False, 
                            square=True, 
                            linewidths=1, linecolor='lightgray')
                
                # 텍스트 추가 (개수 및 비율)
                for i in range(cm_ordered.shape[0]):
                    for j in range(cm_ordered.shape[1]):
                        count = cm_ordered[i, j]
                        total_actual = row_sums[i]
                        ratio = (count / total_actual * 100) if total_actual > 0 else 0
                        text_val = f"{count}\n({ratio:.1f}%)"
                        
                        # 글자색 자동 조정 (배경색에 따라)
                        text_color = "black"
                        if count == 0: 
                            text_color = "red" 
                        elif count > cm_ordered.max() * 0.5: 
                            text_color = "white"
                            
                        ax.text(j + 0.5, i + 0.5, text_val, 
                                ha="center", va="center", 
                                color=text_color, fontsize=16, fontweight='bold')

                # Y축 역순 (Top-Down)
                ax.invert_yaxis()
                
                # 스타일 조정
                ax.tick_params(axis='x', pad=10) 
                ax.tick_params(axis='y', pad=10)
                
                ax.set_yticklabels(final_display_labels, rotation=90, va='center', fontsize=14, fontweight='bold')
                ax.set_xticklabels(final_display_labels, rotation=0, ha='center', fontsize=14, fontweight='bold')
                
                ax.set_xlabel("Predicted cases", fontsize=20, fontweight='bold', labelpad=20)
                ax.set_ylabel("Actual cases", fontsize=20, fontweight='bold', labelpad=20)
                
                # 타이틀 (Accuracy, F1 포함)
                acc = accuracy_score(y_true_raw, y_pred_raw)
                f1 = f1_score(y_true_raw, y_pred_raw, average='weighted')
                display_name = model_pretty_names.get(model_name, model_name)
                
                ax.set_title(f"{display_name}\n(Acc: {acc:.2%} | F1: {f1:.3f})", 
                             fontsize=20, fontweight='bold', pad=20)
                
                # 메모리 정리
                del model
                torch.cuda.empty_cache()
                
            except Exception as e:
                print(f"Error processing {model_name}: {e}")
                ax.text(0.5, 0.5, "Error Loading Model", ha='center', va='center', fontsize=15)
                ax.axis('off')

    # 간격 조정 및 저장
    plt.tight_layout()
    plt.subplots_adjust(wspace=0.4, hspace=0.4)
    
    save_path = os.path.join(FOLDERS['results'], 'confusion_matrices_final_v3.png')
    plt.savefig(save_path, dpi=300)
    plt.show()
    print(f">> 그래프 저장 완료: {save_path}")

# 실행
plot_confusion_matrices_final_v3()

In [ ]:
# [Cell 8] Grad-CAM 전체 이미지 시각화 (Green/Red Text & Yellow Boundary)
# 설명: 모든 모델에 대해 모든 Test 이미지를 Grad-CAM으로 시각화합니다.

# Green Text: 정답 일치
# Red Text: 정답 불일치 (오답)
# Yellow Boundary: 전처리 단계에서 검출된 몸통 라인 표시
# Grad-CAM: 모델이 집중한 부위 (무지개색 히트맵, 붉은색 : 높음, 파란색 : 낮음)

#====================================
# 최소 한번 설치 후 주석처리 요망.
#!pip install grad-cam
#====================================

import gc
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import os

# --- Grad-CAM Hook Class (메모리 최적화 버전) ---
class SimpleGradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.handlers = []
        self.gradients = None
        self.activations = None
        # Inplace 연산 에러 방지
        for m in self.model.modules():
            if hasattr(m, 'inplace'): m.inplace = False
            
        def forward_hook(module, input, output):
            self.activations = output
            output.register_hook(self.save_grads)
        self.handlers.append(target_layer.register_forward_hook(forward_hook))

    def save_grads(self, grad): self.gradients = grad

    def generate(self, input_tensor, target_label):
        self.model.zero_grad()
        output = self.model(input_tensor)
        
        # [메모리 최적화] retain_graph=True 제거 -> VRAM 누수 방지
        output[0, target_label].backward() 
        
        # 데이터 추출 후 즉시 CPU로 이동 및 Detach
        grads = self.gradients.detach().cpu().numpy()[0]
        acts = self.activations.detach().cpu().numpy()[0]
        
        # VRAM에서 참조 해제
        self.gradients = None
        self.activations = None
        
        weights = np.mean(grads, axis=(1, 2))
        cam = np.zeros(acts.shape[1:], dtype=np.float32)
        for i, w in enumerate(weights): cam += w * acts[i, :, :]
        
        cam = np.maximum(cam, 0)
        
        # [수정됨] 해상도 하드코딩(224) 제거 -> IMG_SIZE(448)로 변경
        # IMG_SIZE 변수가 없다면 448로 기본값 설정
        target_size = size if 'size' in globals() else input_tensor.shape[2]
        cam = cv2.resize(cam, (target_size, target_size))
        
        cam = (cam - np.min(cam)) / (np.max(cam) - np.min(cam) + 1e-10)
        
        pred_idx = output.argmax(dim=1).item()
        
        # 출력 텐서 삭제
        del output
        return cam, pred_idx
    
    def close(self):
        for h in self.handlers: h.remove()
        self.handlers = []

# --- 통합 시각화 함수 ---
def visualize_gradcam_safe_with_box():
    print(">> [분석 모드] Grad-CAM + 암 의심 영역(Pink Box) 시각화 시작...")
    
    models_list = ['resnet50', 'densenet121', 'densenet169', 'densenet161']
    test_ds = dataloaders['test'].dataset
    
    def denormalize(tensor):
        mean, std = np.array([0.485, 0.456, 0.406]), np.array([0.229, 0.224, 0.225])
        img = tensor.permute(1, 2, 0).cpu().numpy()
        return np.clip(img * std + mean, 0, 1)

    # [Helper] 히트맵에서 가장 뜨거운 영역(Attention Box) 찾기
    def get_attention_box(cam_mask, threshold=0.7):
        # 상위 30% 강도 이상인 부분 추출
        _, thresh = cv2.threshold(cam_mask, threshold, 1, cv2.THRESH_BINARY)
        thresh = np.uint8(thresh * 255)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if contours:
            c = max(contours, key=cv2.contourArea) # 가장 큰 덩어리
            return cv2.boundingRect(c) # x, y, w, h
        return None

    cols, rows = 6, 4
    num_imgs = len(test_ds)
    
    # tqdm으로 전체 페이지 진행률 표시
    page_iterator = tqdm(range(0, num_imgs, cols), desc="Generating Pages")
    
    for start_idx in page_iterator:
        end_idx = min(start_idx + cols, num_imgs)
        batch_len = end_idx - start_idx
        
        # 그림판 생성 (4행 6열)
        fig, axes = plt.subplots(rows, cols, figsize=(24, 18)) 
        plt.subplots_adjust(hspace=0.4, wspace=0.3)
        
        for m_idx, name in enumerate(models_list):
            try:
                # 모델 로드 (필요할 때만 로드하고 바로 삭제)
                pth_path = os.path.join(FOLDERS['results'], f'best_{name}.pth')
                if not os.path.exists(pth_path): continue
                
                model = ModelFactory.get_model(name, len(class_names))
                model.load_state_dict(torch.load(pth_path, map_location=DEVICE))
                model.eval()
                
                target = model.layer4[-1] if 'resnet' in name else model.features.norm5
                cam_tool = SimpleGradCAM(model, target)
                
                for i in range(batch_len):
                    idx = start_idx + i
                    img_tensor, label = test_ds[idx]
                    input_tensor = img_tensor.unsqueeze(0).to(DEVICE)
                    
                    # Grad-CAM 생성
                    mask, pred = cam_tool.generate(input_tensor, label)
                    
                    # 이미지 합성 (CPU 연산)
                    img_rgb = denormalize(img_tensor)
                    img_uint8 = np.uint8(255 * img_rgb)
                    
                    heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
                    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
                    
                    # [중요] 크기 불일치 방지용 리사이즈 (혹시 모를 오차 대비)
                    if heatmap.shape[:2] != img_rgb.shape[:2]:
                        heatmap = cv2.resize(heatmap, (img_rgb.shape[1], img_rgb.shape[0]))
                    
                    overlay = (heatmap.astype(float)/255 + img_rgb) / 2
                    final_vis = np.uint8(255 * overlay).copy()

                    # ---------------------------------------------------------
                    # [시각화] 노란색 몸통 라인 (Yellow Line)
                    # ---------------------------------------------------------
                    gray_crop = cv2.cvtColor(img_uint8, cv2.COLOR_RGB2GRAY)
                    _, thresh = cv2.threshold(gray_crop, 10, 255, cv2.THRESH_BINARY)
                    contours_body, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    
                    if contours_body:
                        c_body = max(contours_body, key=cv2.contourArea)
                        cv2.drawContours(final_vis, [c_body], -1, (255, 255, 0), 2) # Yellow

                    # ---------------------------------------------------------
                    # [시각화] 암 의심 영역 박스 (Pink Box)
                    # ---------------------------------------------------------
                    # box = get_attention_box(mask)
                    # if box:
                    #    bx, by, bw, bh = box
                    #    cv2.rectangle(final_vis, (bx, by), (bx+bw, by+bh), (255, 0, 255), 2) # Pink

                    # ---------------------------------------------------------
                    # 그래프 그리기
                    # ---------------------------------------------------------
                    ax = axes[m_idx, i]
                    ax.imshow(final_vis)
                    
                    color = 'green' if label == pred else 'red'
                    fname = os.path.basename(test_ds.samples[idx][0])
                    
                    title_text = f"[{name}]\nAct: {class_names[label]}\nPred: {class_names[pred]}"
                    ax.set_title(title_text, color=color, fontsize=14, fontweight='bold')
                    
                    ax.set_xlabel(fname, fontsize=12)
                    ax.set_xticks([]); ax.set_yticks([])
                    
                    del input_tensor
                
                # 모델별 루프 종료 후 정리
                cam_tool.close()
                del model, cam_tool
                torch.cuda.empty_cache() # VRAM 비우기
                
            except Exception as e:
                print(f"!! Error processing {name}: {e}")
                torch.cuda.empty_cache()
            
        # 빈 칸 처리
        if batch_len < cols:
            for i in range(batch_len, cols):
                for j in range(rows): axes[j, i].axis('off')
        
        # 페이지 저장
        page = start_idx // cols + 1
        save_path = os.path.join(FOLDERS['gradcam'], f'GradCAM_Box_Page_{page:03d}.png')
        plt.savefig(save_path, bbox_inches='tight', dpi=150)
        
        # [중요] 메모리 완전 해제
        plt.clf()
        plt.close('all')
        gc.collect() 
        
        # tqdm 업데이트
        page_iterator.set_postfix({'Last Saved': f'Page {page}'})

# 실행
visualize_gradcam_safe_with_box()